In [32]:
#Install Flask and numpy
!pip install flask
!pip install numpy

In [1]:
#create Flask Client 
import flask
import os
import requests

from numpy import genfromtxt
import json
import requests
import torch

In [2]:
url_base = 'http://192.168.50.168:25926/'
response = requests.get(url_base)
response.text

'Base Test is OK'

In [36]:
from numpy import genfromtxt
import json

#prepare dataset
inputs = genfromtxt('my_inputs.csv', delimiter=',', dtype=None)
labels = genfromtxt('my_labels.csv', delimiter=',', dtype=None)
#inputs[:5],labels[:5]

#post dataset to cloud
dict = {'inputs':inputs.tolist(), 'labels':labels.tolist()}
#print(type(dict))

json_dataset = json.dumps(dict )
#print(json_dataset)

url_baseset = 'http://192.168.50.168:25926/dataset'
response = requests.post(url_baseset, json=json_dataset)

response

<Response [200]>

In [37]:
# train model using cloud
url_toTrain = 'http://192.168.50.168:25926/train'
response = requests.get(url_toTrain)
response

<Response [200]>

In [38]:
# get training log
url_log = 'http://192.168.50.168:25926/log'
response = requests.get(url_log)
response.json()

['Epoch: 0 | Loss: 0.94018',
 'Epoch: 100 | Loss: 0.22721',
 'Epoch: 200 | Loss: 0.19472',
 'Epoch: 300 | Loss: 0.17452',
 'Epoch: 400 | Loss: 0.15682',
 'Epoch: 500 | Loss: 0.14419',
 'Epoch: 600 | Loss: 0.13439',
 'Epoch: 700 | Loss: 0.12830',
 'Epoch: 800 | Loss: 0.12175',
 'Epoch: 900 | Loss: 0.11518',
 'Epoch: 1000 | Loss: 0.11613',
 'Epoch: 1100 | Loss: 0.11256',
 'Epoch: 1200 | Loss: 0.11116',
 'Epoch: 1300 | Loss: 0.10269',
 'Epoch: 1400 | Loss: 0.10217',
 'Epoch: 1500 | Loss: 0.10569',
 'Epoch: 1600 | Loss: 0.09556',
 'Epoch: 1700 | Loss: 0.11724',
 'Epoch: 1800 | Loss: 0.08783',
 'Epoch: 1900 | Loss: 0.09002']

In [39]:
# inference using cloud
url_log = 'http://192.168.50.168:25926/inference'
json_inputs = json.dumps({ 'input':[4,2]})
response = requests.post(url_log, json=json_inputs)
response.json()

# output the result
predicts = response.json()
print(predicts)


[0.9221413135528564, 0.02968962863087654, 0.04816901311278343]


In [40]:
# inference using cloud
url_log = 'http://192.168.50.168:25926/inference'
json_inputs = json.dumps({ 'input':[1,8]})
response = requests.post(url_log, json=json_inputs)
response.json()

# output the result
predicts = response.json()
print(predicts)


[1.0, 3.58014284884689e-10, 1.941243177971084e-18]


In [41]:
# inference using cloud
url_log = 'http://192.168.50.168:25926/inference'
json_inputs = json.dumps({ 'input':[8,12]})
response = requests.post(url_log, json=json_inputs)
response.json()

# output the result
predicts = response.json()
print(predicts)


[1.0, 5.8176670690427136e-21, 3.7465468796865626e-31]


In [18]:
# use x_test_input and y_test_label to test the model
x_test_input = genfromtxt('my_test_inputs.csv', delimiter=',', dtype=None)
y_test_label = genfromtxt('my_test_labels.csv', delimiter=',', dtype=None)

# use model to predict(input x_test_input)
url_log = 'http://192.168.50.168:25926/inference'
json_inputs = json.dumps({ 'input':x_test_input.tolist()})
response = requests.post(url_log, json=json_inputs)
predicts = response.json()

# trun predicts to torch tensor
predicts = torch.tensor(predicts)

# trun y_test_label to torch tensor
y_test_label = torch.tensor(y_test_label)

# compare the predict result with the test data
corrects = 0
for i in range(len(predicts)):
    if torch.argmax(predicts[i]) == torch.argmax(y_test_label[i]):
        corrects += 1

accruacy = corrects/len(predicts)*100

# print out the accuracy
print(f'Accuracy: {accruacy:.2f}%')


Accuracy: 93.33%
